In [ ]:
import scrapy
import json
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import xmltodict
import pymongo
class MegatimingSpider(scrapy.Spider):
    name = "MegatimingSpider"
    errors = [];
    def start_requests(self):
        urls = [
            'http://megatiming.pl/contests/regional/1',
            'http://megatiming.pl/contests/regional/2',
            'http://megatiming.pl/contests/regional/3',
            'http://megatiming.pl/contests/regional/4',
            'http://megatiming.pl/contests/regional/5',
            'http://megatiming.pl/contests/regional/6',
            'http://megatiming.pl/contests/regional/7',
            'http://megatiming.pl/contests/regional/8',
            'http://megatiming.pl/contests/regional/9',
            'http://megatiming.pl/contests/regional/10',
            'http://megatiming.pl/contests/regional/11',
            'http://megatiming.pl/contests/regional/12',
            'http://megatiming.pl/contests/regional/13',
            'http://megatiming.pl/contests/regional/14',
            'http://megatiming.pl/contests/regional/15',
            'http://megatiming.pl/contests/regional/16',
            'http://megatiming.pl/contests/regional/17',
            'http://megatiming.pl/contests/regional/18',
            'http://megatiming.pl/contests/regional/19',
            'http://megatiming.pl/contests/regional/20',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        js_script = response.xpath('/html/body/script[contains(., "contests")]/text()').get()
        json_source=js_script[js_script.find('data=')+6:-1]       
        json_source_1 =  json_source[json_source.find('"contests"')+11:] 
        json_source_2 = json_source_1[:json_source_1.find('"home"')-1];
        data = json.loads(json_source_2)
        for key in data["data"]["contests"].keys():
            #print(key)
            yield scrapy.Request(url="http://megatiming.pl/contest/"+key,
                             callback=self.parseContest,
                             cb_kwargs=dict(uuid=key))
        
    def parseContest(self,response,uuid):
        fileName = self.getFileName(response,uuid)
        if (fileName):
            self.writeToDatabase(fileName,uuid)
    
    def getFileName(self,response,uuid):
        try:
            js_script = response.xpath('/html/body/script[contains(., "contests")]/text()').get()
            json_source_1=js_script[js_script.find('"contests"')+11:-1]
            json_source_2=json_source_1[:json_source_1.find('"home"')-1]
            data = json.loads(json_source_2)
            for element in data["data"]["contests"][uuid]["files"]["result"]:
                if (element["url"].find("lxf")>=0):
                    return(element["url"])
            return None
        except:
            return None
    
    def writeToDatabase(self,url,uuid):
        try:
            resp = urlopen(url)
            zipfile = ZipFile(BytesIO(resp.read()))
            nameFileLef = zipfile.namelist()[0]
            xmlFile = zipfile.read(nameFileLef).decode("utf-8") 
        
            # change wrong name attributes
            #result.url, organizer.url, hostclub.url -> result_url , organizer_url , hostclub_url
            xmlFile = xmlFile.replace("result.url","result_url");
            xmlFile = xmlFile.replace("organizer.url","organizer_url");
            xmlFile = xmlFile.replace("hostclub.url","hostclub_url");
            xmlFile = xmlFile.replace("shortname.en","shortname_en")
            xmlFile = xmlFile.replace("firstname.en","firstname_en")
            xmlFile = xmlFile.replace("name.en","name_en")
            xmlFile = xmlFile.replace("lastname.en","lastname_en")
            xmlFile = xmlFile.replace("city.en","city_en")
        
            # read to dictionary
            data = xmltodict.parse(xmlFile)
            data["uuid"] = uuid
            # open mongodb and write dictionary
            myclient = pymongo.MongoClient("mongodb://localhost:27017/")
            mydb = myclient["megatiming"]
            mycol = mydb["results"]
            mycol.insert_one(data)
       
        except Exception as e:
            self.errors.append(uuid+" error "+str(e))
    

In [ ]:
from scrapy.crawler import CrawlerProcess
process = CrawlerProcess()
spider = MegatimingSpider
process.crawl(spider)
process.start()

In [ ]:
spider.errors